In [28]:
import ast
import io
import json
import tokenize
from typing import Tuple, List, Any
import re

def extract_and_parse_prompt_list(code: str) -> List[Tuple[str, ...]]:
    """
    From a string of Python code, finds the first occurrence of
        = [ ... ]
    and parses that bracketed literal into a List[Tuple[str, ...]].

    Raises:
        ValueError if no list literal is found or it’s malformed.
    """
    # 1) grab everything from the first '=' up to the matching ']'
    m = re.search(r'=\s*(\[\s*[\s\S]*?\])', code)
    if not m:
        raise ValueError("No list literal found after an '=' in the code")
    list_str = m.group(1)

    # 2) safely evaluate it (only literals)
    try:
        data: Any = ast.literal_eval(list_str)
    except (SyntaxError, ValueError) as e:
        raise ValueError(f"Malformed list literal: {e}")

    # 3) validate shape
    if not isinstance(data, list) or not all(
        isinstance(item, (list, tuple)) and all(isinstance(x, str) for x in item) for item in data
    ):
        raise ValueError(
            "Parsed object is not a list of tuples/lists of strings"
        )

    # 4) convert to List[Tuple[str, ...]]
    return [tuple(str(x) for x in item) for item in data]


def _fix_quote_issues(code: str) -> str:
    """
    Fix common quote issues in LLM-generated code.
    """
    import re
    
    # Pattern to find single quotes around words inside string literals
    # This converts 'indistinct' to "indistinct"
    fixed_code = re.sub(r"'([a-zA-Z]+)'", r'"\1"', code)
    
    # Pattern to find apostrophes in contractions/possessives
    # This converts cluster's to cluster"s
    fixed_code = re.sub(r"([a-zA-Z]+)'([a-zA-Z]*)", r'\1"\2', fixed_code)
    
    # Finally, swap all single quotes to double and double quotes to single
    # Use a placeholder to avoid conflicts during replacement
    placeholder = "___TEMP_QUOTE___"
    fixed_code = fixed_code.replace('"', placeholder)  # double -> placeholder
    fixed_code = fixed_code.replace("'", '"')          # single -> double
    fixed_code = fixed_code.replace(placeholder, "'")  # placeholder -> single
    
    return fixed_code


forced_double_str = extract_and_parse_prompt_list("""
list = [('Granules are large, irregular, and exhibit metachromatic staining, shifting from blue to purple depending on the stain and angle of light.', 'Granules are consistently round and uniform, approximately 2-4 μm in diameter, and stain a bright eosinophilic orange-red, appearing highly refractile and crystalline.', 'Cytoplasm is clear and devoid of granules, presenting a smooth, homogenous appearance and a high nuclear-cytoplasmic ratio.', 'Granules are fine and dust-like, appearing as a faint haze within the cytoplasm, staining a pale lilac color and being sparsely distributed.', 'Granules are small and round, evenly distributed throughout the cytoplasm, creating a finely granular texture and a pale pink hue, giving a "ground glass" appearance.'),
('Granules are large, irregular, and exhibit metachromasia, shifting from blue to purple depending on the stain and angle of light.', 'Granules are consistently round and uniform, approximately 2-4 μm in diameter, and stain a bright eosinophilic orange-red, appearing highly refractile and crystalline.', 'Cytoplasm is clear and devoid of granules, presenting a smooth, homogenous appearance and a high nuclear-cytoplasmic ratio.', 'Granules are fine and dust-like, appearing as a faint haze within the cytoplasm, staining a pale lilac color and being sparsely distributed.', "Granules are small and round, evenly distributed throughout the cytoplasm, creating a finely granular texture and a pale pink hue, giving a 'ground glass' appearance."),
('Granules are large, irregular, and exhibit metachromasia, shifting from blue to purple-red depending on the angle of light.', 'Granules are consistently round and uniform, approximately 2-3 μm in diameter, and stain intensely with eosin, exhibiting a bright orange-red color.', 'Cytoplasm is devoid of granules, presenting a clear and homogenous appearance with a smooth texture and minimal staining.', 'Granules are fine and dust-like, appearing as a faint haze within the cytoplasm, staining a pale lilac color.', 'Granules are small and round, evenly distributed throughout the cytoplasm, creating a finely granular texture and a pale pink hue.'),
('Nuclear shape is often irregular and convoluted due to the pressure from abundant granules.', 'Nuclear shape is consistently round to oval, with a smooth nuclear membrane.', 'Nuclear shape is round and centrally located, with a smooth, uniform contour.', 'Nuclear shape is often kidney-bean shaped or horseshoe-shaped, with a prominent nucleolus.', 'Nuclear shape is segmented, with 3-5 distinct lobes connected by thin strands of chromatin.'),
('Cytoplasm is characterized by a dense concentration of large, dark purple granules (1-2 μm) exhibiting metachromasia, often obscuring the nucleus.', 'Cytoplasm is filled with numerous, large (2-5 μm), refractile, orange-red granules containing a central crystalline core.', 'Cytoplasm is minimal, appearing as a thin, pale blue rim around a densely packed, round nucleus; granules are absent.', "Cytoplasm is abundant, grey-blue, and contains numerous fine, evenly distributed azurophilic granules, giving a 'ground glass' appearance.", 'Cytoplasm is pale pink and densely filled with numerous, small (0.5-1 μm), neutral granules, creating a finely granular texture.'),
('Cytoplasm is intensely basophilic, appearing a deep violet-blue and almost completely obscuring the nucleus.', 'Cytoplasm exhibits a bright coral-pink hue due to abundant, large eosinophilic granules.', 'Cytoplasm is scant and pale blue, presenting a clear appearance.', 'Cytoplasm is abundant, dull grey-blue, and contains numerous fine granules.', 'Cytoplasm is pale peach-colored and densely packed with numerous granules.'),
("Cytoplasm is densely packed with dark purple granules (1-2 μm) exhibiting metachromasia, often obscuring the nucleus and creating a 'smudge' effect.", 'Cytoplasm is filled with numerous, large (2-5 μm), refractile, orange-red granules containing a central crystalline core, imparting a brilliant sheen.', 'Cytoplasm is minimal, appearing as a thin, pale blue rim around a densely packed, round nucleus; granules are entirely absent.', "Cytoplasm is abundant, grey-blue, and contains numerous fine, evenly distributed azurophilic granules, giving a 'ground glass' appearance.", 'Cytoplasm is pale pink and densely filled with numerous, small (0.5-1 μm), neutral granules, creating a finely granular texture and a slightly hazy appearance.'),
('Granules are intensely basophilic, staining a deep violet color, and are often arranged in irregular clusters, creating a heterogeneous cytoplasmic appearance.', 'Granules are consistently spherical, 2-4 μm in diameter, and intensely eosinophilic, appearing as bright orange-red spheres with a distinct refractive index.', 'Cytoplasm is minimal and clear, with a high nuclear-cytoplasmic ratio and a faint blue tint; no granules are present.', 'Cytoplasm is greyish-blue, with a slightly granular texture and occasional small, clear vacuoles; granules are less than 0.5 μm.', 'Cytoplasm is pale pink, with a finely granular texture and a uniform distribution of neutral granules; granules are 0.5-1 μm.'),
('Granules are large and irregularly shaped, often exhibiting metachromasia, shifting from blue to purple, and creating a heterogeneous cytoplasmic appearance.', 'Granules are consistently round and uniform, approximately 2-3 μm in diameter, and stain intensely with eosin, exhibiting a bright orange-red color.', 'Cytoplasm is devoid of granules, presenting a clear and homogenous appearance with a smooth texture and minimal staining.', 'Granules are fine and dust-like, appearing as a faint haze within the cytoplasm, staining a pale lilac color.', 'Granules are small and round, evenly distributed throughout the cytoplasm, creating a finely granular texture and a pale pink hue.'),
('Granule morphology is highly variable, ranging from round to irregular and often clumped together, with a dark purple staining intensity.', 'Granule morphology is consistently round and uniform, with a diameter of 1-2 μm, and exhibits a bright orange-red color.', 'Granules are absent, resulting in a homogenous cytoplasm with no visible inclusions or texture, and a prominent, centrally located nucleus.', 'Granule morphology is fine and dust-like, sparsely distributed throughout the cytoplasm, staining a pale lilac color.', 'Granule morphology is small and round, evenly dispersed throughout the cytoplasm, creating a finely granular texture and a pale pink hue.'),
('Cytoplasm is densely packed with large, dark purple metachromatic granules that often obscure the nucleus.', 'Cytoplasm contains numerous, large, refractile eosinophilic granules with a distinct core, appearing bright orange-red.', "Cytoplasm is sparse and clear, with occasional small, pale blue granules; the nucleus dominates the cell's appearance.", 'Cytoplasm is abundant, greyish-blue, and contains numerous fine, lilac-colored azurophilic granules distributed throughout.', "Cytoplasm is pale pink and filled with numerous, small, neutral granules that create a 'sand-like' texture; the nucleus is segmented."),
("Granules are large, dark purple, and exhibit metachromasia, often obscuring the nucleus and creating a 'smudged' appearance.", 'Granules are bright orange-red, refractile, and contain a crystalline core, often forming clusters and extending to the cell periphery.', 'Granules are absent, leaving the cytoplasm clear and homogenous with a high nuclear-cytoplasmic ratio.', "Granules are fine and evenly dispersed, creating a subtle, dusty appearance and a 'ground glass' cytoplasm.", 'Granules are small, round, and densely packed, giving the cytoplasm a slightly granular texture and a segmented nucleus.'),
('Cytoplasm is intensely stained with metachromatic granules, appearing dark blue-purple and completely obscuring the nucleus; granules are 1-3 μm in diameter and irregularly shaped.', 'Cytoplasm is filled with large, intensely eosinophilic granules that are spherical and highly refractile, creating a bright orange-red appearance; granules measure 2-5 μm.', 'Cytoplasm is scant and clear, with a smooth texture and a pale blue tint; the nucleus dominates the cell volume and is deeply stained.', 'Cytoplasm is abundant, greyish-blue, and contains numerous small, clear vacuoles and a fine, dust-like sprinkling of azurophilic granules.', "Cytoplasm is pale pink and densely packed with small, evenly distributed neutral granules, giving a 'ground glass' appearance; granules are approximately 0.5-1 μm in diameter."),
('Cytoplasm is densely packed with large, dark purple granules exhibiting metachromasia, often obscuring the nucleus.', 'Cytoplasm contains numerous, uniformly distributed, bright orange-red granules, extending to the cell periphery and surrounding the bilobed nucleus.', 'Cytoplasm is minimal, with a high nuclear-cytoplasmic ratio and a clear, pale blue appearance, lacking visible granules.', "Cytoplasm is abundant, grey-blue, and contains numerous fine, azurophilic granules and occasional vacuoles, giving a 'ground glass' appearance.", 'Cytoplasm is pale pink and filled with numerous, evenly distributed, lilac-colored granules, with a slightly granular texture and a segmented nucleus.'),
('Granules are irregularly shaped and intensely basophilic, staining a deep violet color and often obscuring the cell boundaries.', 'Granules are spherical and intensely eosinophilic, appearing as bright orange-red spheres throughout the cytoplasm; granules are 1-3 μm in diameter.', 'Granules are virtually absent, resulting in a clear cytoplasm and a prominent, densely stained nucleus with condensed chromatin.', 'Granules are few, small, and azurophilic, appearing as a faint, greyish-purple dusting within the cytoplasm.', 'Granules are small, uniform, and neutrally stained, creating a slightly granular cytoplasmic background; granules are 0.2-0.5 μm in diameter.'),
("Cytoplasm is almost entirely filled with dark purple, metachromatic granules, obscuring the nucleus and creating a 'solid' appearance.", 'Cytoplasm is a vibrant orange-red due to the abundance of eosinophilic granules, which are often clustered together.', 'Cytoplasm is minimal and clear, with a smooth texture and a high nuclear-cytoplasmic ratio; the cytoplasm stains a pale, delicate blue.', "Cytoplasm is greyish-blue, with a slightly granular texture and occasional small, clear vacuoles that appear as 'empty spaces'.", 'Cytoplasm is pale pink, with a finely granular texture and a uniform distribution of neutral granules; the granules are readily visible but do not mask the nucleus.'),
('Cytoplasm is intensely stained with metachromatic granules, appearing dark blue-purple and often completely obscuring the nucleus; granules are 1-3 μm in diameter and irregularly shaped.', 'Cytoplasm is filled with large, intensely eosinophilic granules that are spherical and highly refractile, creating a bright orange-red appearance; granules measure 2-5 μm.', 'Cytoplasm is scant and clear, with a smooth texture and a pale blue tint; the nucleus dominates the cell volume.', 'Cytoplasm is abundant, greyish-blue, and contains numerous small, clear vacuoles and a fine, dust-like sprinkling of azurophilic granules.', "Cytoplasm is pale pink and densely packed with small, evenly distributed neutral granules, giving a 'ground glass' appearance; granules are approximately 0.5-1 μm in diameter."),
("Basophil cytoplasm is densely filled with irregular, dark blue-purple granules that often mask the nucleus, exhibiting prominent metachromasia and creating a 'smudged' cellular appearance.", 'Eosinophil cytoplasm is packed with large, spherical granules exhibiting a bright orange-red hue and a distinct, glassy sheen; granules frequently contain a central crystalline inclusion.', 'Lymphocyte cytoplasm is scant and pale blue, presenting a smooth, homogenous texture with no visible granules; the nucleus is deeply basophilic and occupies the majority of the cell volume.', "Monocyte cytoplasm is abundant, grey-blue, and contains numerous fine, dust-like azurophilic granules, along with occasional larger vacuoles that impart a 'ground glass' appearance.", 'Neutrophil cytoplasm is pale pink and contains a high density of small, lilac-colored neutral granules, creating a finely granular texture; the segmented nucleus exhibits 3-5 lobes connected by thin chromatin filaments.'),
('Basophil cytoplasm demonstrates a marked absence of vacuoles, appearing densely packed with granules that exhibit metachromasia.', 'Eosinophil cytoplasm contains occasional small, clear vacuoles, often near the cell periphery, alongside bright orange-red granules.', 'Lymphocyte cytoplasm is minimal and devoid of vacuoles, appearing as a thin rim around the nucleus.', 'Monocyte cytoplasm is abundant and frequently contains prominent vacuoles, sometimes with ingested material.', 'Neutrophil cytoplasm may exhibit occasional small, clear vacuoles, but these are less frequent than in monocytes.'),
('Cytoplasm exhibits a pronounced metachromatic effect, shifting from deep blue to reddish-purple depending on the stain and viewing angle.', 'Cytoplasm is intensely eosinophilic, with granules appearing bright orange-red and exhibiting a crystalline core under high magnification.', 'Cytoplasm is scant and pale blue, with a smooth texture and a high nuclear-cytoplasmic ratio.', "Cytoplasm is abundant, grey-blue, and contains numerous fine, evenly distributed azurophilic granules, giving a 'frosted glass' appearance.", 'Cytoplasm is pale pink and densely packed with small, neutrally stained granules, giving it a finely granular texture.'),
('Granules are irregular in shape and size, staining intensely with basic dyes and often forming dense clusters.', 'Granules are uniform in size and shape, appearing as bright orange-red spheres throughout the cytoplasm, often refractile.', 'Granules are absent, resulting in a clear cytoplasm and a prominent, deeply stained nucleus with a high nuclear-cytoplasmic ratio.', 'Granules are sparse and fine, appearing as a faint greyish-purple dusting concentrated near the cell periphery, often indistinct.', 'Granules are small and evenly distributed, staining a neutral pink and providing a slightly granular cytoplasmic background.'),
('Granules are highly polymorphic, varying in size and shape, and staining intensely with basic dyes.', 'Granules are consistently round and uniform, approximately 2-3 μm in diameter, and stain intensely with eosin, exhibiting a bright orange-red color.', 'Cytoplasm is devoid of granules, presenting a clear and homogenous appearance with a smooth texture.', 'Granules are fine and dust-like, appearing as a faint haze within the cytoplasm, staining a pale lilac color.', 'Granules are small and round, evenly distributed throughout the cytoplasm, creating a finely granular texture and a pale pink hue.'),
("Basophil granules are often so numerous they distort the cell's shape, creating an irregular outline.", 'Eosinophil granules are typically 2-3 μm in diameter and are evenly distributed throughout the cytoplasm, creating a homogenous orange-red appearance.', 'Lymphocyte nucleus occupies >80% of the cell volume, leaving minimal cytoplasm for feature assessment.', 'Monocyte cytoplasm contains pseudopodia, giving the cell an amoeboid shape and indicating active motility.', 'Neutrophil granules are non-lysosomal and contain enzymes involved in phagocytosis and inflammation.'),
('Cytoplasm is densely filled with large, irregular, dark purple granules that exhibit metachromatic staining, shifting from blue to reddish-purple, and often obscure the nucleus.', "Cytoplasm is abundant and packed with large, intensely orange-red granules containing a central crystalline core and surrounded by a clear halo, giving a 'sparkling' appearance.", 'Cytoplasm is scant, clear, and pale blue, with a high nuclear-cytoplasmic ratio (9:1) and a round, deeply basophilic nucleus exhibiting smooth, condensed chromatin.', "Cytoplasm is broad, grey-blue, and contains numerous small, lilac-colored azurophilic granules, along with occasional clear vacuoles creating a 'ground glass' effect.", 'Cytoplasm is pale pink and contains numerous, evenly distributed, fine lilac granules (0.5-1 μm) and a segmented nucleus with 3-5 lobes connected by thin chromatin strands.'),
("Cytoplasm is densely packed with large, irregular, dark purple granules that often obscure the nucleus, exhibiting metachromatic staining and creating a 'smudge cell' effect.", 'Cytoplasm is filled with numerous, large (2-5 μm), refractile, orange-red granules containing a central crystalline core, imparting a brilliant sheen.', 'Cytoplasm is minimal, appearing as a thin, pale blue rim around a densely packed, round nucleus; granules are entirely absent.', "Cytoplasm is abundant, grey-blue, and contains numerous fine, evenly distributed azurophilic granules, giving a 'ground glass' appearance.", 'Cytoplasm is pale pink and densely filled with numerous, small (0.5-1 μm), neutral granules, creating a finely granular texture and a slightly hazy appearance.'),
("Cytoplasm is densely packed with large, irregular, dark purple granules that often obscure the nucleus, exhibiting metachromasia and creating a 'smudge cell' effect, with granules ranging from 1.2-1.8 μm.", 'Cytoplasm is filled with numerous, large (2-5 μm), refractile, orange-red granules containing a central crystalline core, imparting a brilliant sheen.', 'Cytoplasm is minimal, appearing as a thin, pale blue rim around a densely packed, round nucleus; granules are entirely absent, and the cell measures 7-10 μm in diameter.', "Cytoplasm is abundant, grey-blue, and displays a 'frosted glass' appearance due to numerous, small (0.5-1 μm) vacuoles and sparse, lilac-colored azurophilic granules.", "Cytoplasm is packed with numerous, small (0.5-1 μm), pale lilac granules evenly distributed throughout, creating a finely granular 'ground glass' effect, and the cell measures 10-12 μm."),
('Cytoplasm is densely packed with metachromatic granules, appearing almost black in stained preparations, and granules are 1-2 μm in diameter.', 'Cytoplasm is a brilliant orange-red, with granules exhibiting a glassy, refractive quality and measuring 3-6 μm.', 'Cytoplasm is minimal and clear, with a high nuclear-cytoplasmic ratio and a faint blue tint; no granules are present.', 'Cytoplasm is greyish-blue, with a slightly granular texture and occasional small, clear vacuoles; granules are less than 0.5 μm.', 'Cytoplasm is pale pink, with a finely granular texture and a uniform distribution of neutral granules; granules are 0.5-1 μm.'),
('Granules are irregularly shaped and vary in size, creating a heterogeneous cytoplasmic appearance with a dark purple hue and a tendency to clump.', 'Granules are consistently spherical and approximately 2-4 μm in diameter, staining a bright eosinophilic orange-red and evenly distributed.', 'Granules are absent, resulting in a clear cytoplasm with a smooth texture and minimal staining, highlighting the prominent nucleus.', 'Granules are small, sparse, and stain a faint lilac color, appearing as subtle inclusions within the grey-blue cytoplasm, concentrated peripherally.', "Granules are uniformly sized and densely packed, giving the cytoplasm a 'ground glass' appearance with a pale pink tint and a segmented nucleus."),
('Cytoplasm is densely packed with large, irregularly shaped, dark violet granules that often obscure the nucleus, exhibiting strong metachromasia.', 'Cytoplasm is filled with abundant, large, and highly refractile orange-red granules, often forming distinct clusters and extending to the cell membrane.', 'Cytoplasm is minimal, almost absent, resulting in a high nuclear-cytoplasmic ratio and a clear, pale blue appearance; no visible granules are present.', "Cytoplasm is broad and greyish-blue, containing numerous fine, evenly distributed azurophilic granules and prominent, clear vacuoles giving a 'frosted glass' appearance.", 'Cytoplasm is pale pink and contains numerous small, evenly distributed neutral granules, creating a finely granular texture; granules are typically 1-2 μm in diameter.'),
("Granules are large, dark purple, and exhibit metachromasia, often obscuring the nucleus and creating a 'smudged' appearance, with irregular shapes.", 'Granules are bright orange-red, refractile, and contain a crystalline core, often forming clusters and extending to the cell periphery.', 'Granules are absent, leaving the cytoplasm clear and homogenous with a high nuclear-cytoplasmic ratio and a smooth nuclear membrane.', "Granules are fine and evenly dispersed, creating a subtle, dusty appearance and a 'ground glass' cytoplasm, staining pale lilac.", 'Granules are small, round, and densely packed, giving the cytoplasm a slightly granular texture and a segmented nucleus.'),
('Cytoplasm is densely packed with metachromatic granules, appearing almost black in stained preparations, and granules are 0.8-1.5 μm in diameter.', 'Cytoplasm is a brilliant orange-red, with granules exhibiting a glassy, refractive quality and measuring 3-6 μm.', 'Cytoplasm is minimal and clear, with a high nuclear-cytoplasmic ratio and a faint blue tint; no granules are present.', 'Cytoplasm is greyish-blue, with a slightly granular texture and occasional small, clear vacuoles; granules are less than 0.5 μm.', 'Cytoplasm is pale pink, with a finely granular texture and a uniform distribution of neutral granules; granules are 0.5-1 μm.'),
("Basophil cell shape is often irregular and distorted due to the numerous granules, appearing 'ruffled'.", 'Eosinophil cell shape is generally round to oval, maintaining a relatively consistent outline.', 'Lymphocyte cell shape is typically round and uniform, with a smooth cell membrane.', 'Monocyte cell shape is highly variable and amoeboid, frequently changing shape as it moves.', 'Neutrophil cell shape is round to slightly elongated, conforming to the shape of the segmented nucleus.'),
('Basophil granules are large, dark purple, and exhibit metachromasia, shifting to a reddish-violet hue upon staining, often obscuring the nucleus.', 'Eosinophil granules are consistently spherical, 2-4 μm in diameter, and exhibit a bright orange-red color with a glassy appearance and a crystalline core.', 'Lymphocyte cytoplasm is scant and clear, with a high nuclear-cytoplasmic ratio and a nucleus that is round and deeply stained, with condensed chromatin.', 'Monocyte cytoplasm is abundant, grey-blue, and contains a few large, irregular azurophilic granules and numerous vacuoles.', "Neutrophil granules are numerous, pale lilac, and evenly dispersed, creating a characteristic 'sandpaper' appearance with a segmented nucleus."),
("Cell size is small (6-8 μm) with intensely basophilic granules that often mask the nucleus, exhibiting a 'smudge cell' appearance and metachromatic staining with dilute acid dyes.", 'Cell size is 10-14 μm, displaying bilobed nuclei connected by a major groove, and cytoplasm packed with large, refractile, orange-red granules containing a central crystalline core.', 'Cell size is 7-10 μm, characterized by a high nuclear-cytoplasmic ratio (8:1) and a densely packed, round nucleus with smooth chromatin; cytoplasm is a narrow, pale blue rim.', 'Cell size is 12-18 μm, featuring a kidney-shaped nucleus with a loose chromatin pattern and abundant grey-blue cytoplasm containing numerous small vacuoles and fine azurophilic granules.', 'Cell size is 10-12 μm, exhibiting a segmented nucleus (typically 3-5 lobes) connected by thin chromatin strands, and pale lilac granules that are evenly distributed and approximately 0.5-1 μm in diameter.'),
('Cytoplasm exhibits a pronounced metachromatic effect, shifting from deep blue to reddish-purple depending on the stain and viewing angle, and contains numerous irregularly shaped granules.', 'Cytoplasm is intensely eosinophilic, with granules appearing bright orange-red and exhibiting a crystalline core under high magnification, and a smooth cytoplasmic texture.', 'Cytoplasm is scant and pale blue, with a smooth texture and a high nuclear-cytoplasmic ratio, lacking any visible granules or cytoplasmic inclusions.', "Cytoplasm is abundant, grey-blue, and contains numerous fine, evenly distributed azurophilic granules, giving a 'frosted glass' appearance and occasional vacuoles.", 'Cytoplasm is pale pink and densely packed with small, neutrally stained granules, giving it a finely granular texture and a slightly hazy appearance, with a diameter of 1-2 μm.'),
('Cytoplasm color is a deep, rich purple due to the high concentration of metachromatic granules.', 'Cytoplasm color is a vibrant orange-red, reflecting the abundance of eosinophilic granules.', 'Cytoplasm color is a pale, almost transparent blue, with minimal staining.', 'Cytoplasm color is a muted grey-blue, with a slightly granular appearance.', 'Cytoplasm color is a delicate pale pink, with a subtle granular texture.'),
("Cytoplasm is densely packed with large, dark-purple metachromatic granules, often obscuring the nuclear detail and creating a 'smudged' appearance.", 'Cytoplasm contains numerous, large, refractile eosinophilic granules with a distinct core, appearing bright orange-red and exhibiting strong birefringence.', 'Cytoplasm is sparse and clear, with a high nuclear-cytoplasmic ratio and a round, densely stained nucleus, lacking any visible granules or inclusions.', "Cytoplasm is abundant, greyish-blue, and contains numerous fine, lilac-colored azurophilic granules, giving a 'ground glass' appearance with occasional vacuoles.", "Cytoplasm is pale pink and filled with numerous, small, neutral granules, creating a 'sand-like' texture and a segmented nucleus with clearly defined lobes."),
('Granule size is variable, ranging from small dust-like particles to large, prominent structures.', 'Granules are uniformly sized and spherical, with a bright eosinophilic stain.', 'Granules are virtually absent, resulting in a clear, homogenous cytoplasm.', 'Granules are fine and evenly dispersed, creating a subtle, dusty appearance.', 'Granules are small, round, and densely packed, giving the cytoplasm a slightly granular texture.'),
('Granules are variable in size and shape, ranging from small dust-like particles to large, prominent structures.', 'Granules are uniformly sized and spherical, with a bright eosinophilic stain.', 'Granules are virtually absent, resulting in a clear, homogenous cytoplasm.', 'Granules are fine and evenly dispersed, creating a subtle, dusty appearance.', 'Granules are small, round, and densely packed, giving the cytoplasm a slightly granular texture.'),
('Granules are large, dark-blue, and irregularly shaped, often coalescing to obscure the nucleus, exhibiting metachromasia.', 'Granules are bright orange-red, uniform in size, and exhibit a characteristic rod-shaped morphology, containing major basic protein.', 'Granules are absent, resulting in a clear, homogenous cytoplasm and a high nuclear-cytoplasmic ratio.', 'Granules are sparse, fine, and azurophilic, appearing as a faint dusting within the cytoplasm, staining a pale lilac color.', 'Granules are small, neutral, and evenly dispersed, giving the cytoplasm a slightly granular texture and a pale pink hue.'),
('Granules are large and irregular, often exhibiting a metachromatic staining pattern.', 'Granules are uniform in size and shape, exhibiting a characteristic rod-shaped morphology.', 'Granules are absent, resulting in a homogenous, clear cytoplasm.', 'Granules are fine and dust-like, appearing as a faint haze.', 'Granules are small and round, evenly distributed throughout the cytoplasm.'),
('Granules are large, irregular, and exhibit metachromatic staining, shifting from blue to purple depending on the stain and angle of light; granules are 2-5 μm.', 'Granules stain brightly eosinophilic, exhibiting a characteristic red-orange color and appearing as distinct, crystalline inclusions.', 'Granules are absent, and the cytoplasm stains weakly with Romanowsky stains, appearing clear and homogenous.', 'Granules stain faintly azurophilic, appearing as a greyish-purple dusting and being sparsely distributed.', 'Granules stain neutrally, appearing pale pink or lavender and being evenly dispersed throughout the cytoplasm.'),
('Cytoplasm-Color is a deep violet-purple, often with a bluish tinge due to metachromasia.', 'Cytoplasm-Color is a vibrant orange-red, reflecting the high concentration of eosinophilic granules.', 'Cytoplasm-Color is a pale blue, almost transparent, with minimal staining.', 'Cytoplasm-Color is a greyish-blue, with a subtle lilac hue from the azurophilic granules.', 'Cytoplasm-Color is a pale pink, with a slightly hazy appearance due to the neutral granules.'),
('Granules are intensely metachromatic, staining dark purple with thionin and blue-black with toluidine blue.', 'Granules are uniformly eosinophilic, exhibiting a characteristic crystalline core when viewed under polarized light.', 'Granules are absent, resulting in a homogenous, clear cytoplasm with no visible inclusions.', 'Granules are sparse, fine, and azurophilic, appearing as a faint greyish-purple dusting.', 'Granules are small, neutral, and evenly dispersed, giving the cytoplasm a slightly granular appearance; granules do not obscure the nucleus.'),
('Basophil cytoplasm is densely filled with metachromatic granules, appearing almost black in stained preparations, and obscuring cellular details; granules are 1.5-2.5 μm.', 'Eosinophil cytoplasm is a brilliant orange-red, with granules exhibiting a glassy, refractive quality and a consistent spherical morphology; granules are 2-4 μm.', 'Lymphocyte cytoplasm is minimal and clear, with a pale blue tint and a high nuclear-cytoplasmic ratio, devoid of granules; nucleus is round and hyperchromatic.', "Monocyte cytoplasm is greyish-blue, with a slightly granular texture and occasional small, clear vacuoles, creating a 'smudged' appearance; granules are sparse and lilac.", 'Neutrophil cytoplasm is pale pink, with a finely granular texture and a uniform distribution of neutral granules, creating a hazy appearance; nucleus is segmented.'),
('Granules are irregularly shaped and intensely basophilic, staining a deep violet color and often obscuring the cell boundaries.', 'Granules are spherical and intensely eosinophilic, appearing as bright orange-red spheres throughout the cytoplasm; granules are 1-3 μm in diameter.', 'Granules are virtually absent, resulting in a clear cytoplasm and a prominent, densely stained nucleus with condensed chromatin.', 'Granules are sparse, fine, and dust-like, appearing as a faint greyish-purple haze concentrated near the cell periphery.', 'Granules are small, uniform, and neutrally stained, creating a slightly granular cytoplasmic background; granules are 0.2-0.5 μm in diameter.'),
('Granules are variable in size and shape, ranging from small dust-like particles to large, prominent structures, staining intensely with basic dyes.', 'Granules are uniformly sized and spherical, with a bright eosinophilic stain, often arranged in clusters.', 'Cytoplasm is virtually absent, resulting in a clear, homogenous cytoplasm.', 'Granules are fine and evenly dispersed, creating a subtle, dusty appearance.', 'Granules are small, round, and densely packed, giving the cytoplasm a slightly granular texture.'),
('Nucleus is often obscured by densely packed, dark purple granules that exhibit metachromatic staining, appearing blue-black with Giemsa.', 'Nucleus is typically bilobed, with bright orange-red granules containing a crystalline core that refracts light.', 'Nucleus is large, round, and hyperchromatic, occupying most of the cell volume, with a thin rim of pale blue cytoplasm.', 'Nucleus is kidney-shaped or horseshoe-shaped, with a loose chromatin network and a prominent nucleolus; cytoplasm is abundant and grey-blue.', 'Nucleus is segmented into 3-5 lobes connected by thin chromatin filaments; lobes are generally uniform in size and shape.'),
("Basophil nucleus is often hidden by a dense, dark purple granular mass, exhibiting a 'smudge cell' appearance and lacking distinct lobulation.", 'Eosinophil granules are consistently bright orange-red, displaying a central crystalline core and a tendency to cluster near the cell membrane.', 'Lymphocyte nucleus is hyperchromatic and occupies nearly the entire cell volume, with a thin rim of pale blue cytoplasm and no discernible granules.', 'Monocyte cytoplasm is abundant, grey-blue, and contains numerous fine, dust-like azurophilic granules, along with prominent, clear vacuoles.', 'Neutrophil nucleus exhibits a segmented pattern of 3-5 lobes connected by thin chromatin strands, with pale lilac granules evenly distributed.'),
('Nuclear-cytoplasmic ratio is low (1:3 or greater) due to the abundant cytoplasm filled with large, dark granules.', 'Nuclear-cytoplasmic ratio is moderate (1:1 to 1:2) with a clearly defined nucleus and a cytoplasm filled with eosinophilic granules.', 'Nuclear-cytoplasmic ratio is very high (7:1 or greater) with a large, intensely stained nucleus and minimal cytoplasm.', 'Nuclear-cytoplasmic ratio is low (1:2 or greater) with a large, pale nucleus and abundant, grey-blue cytoplasm.', 'Nuclear-cytoplasmic ratio is moderate (2:1) with a segmented nucleus and a pale pink cytoplasm filled with fine granules.'),
("Basophil cytoplasm exhibits a 'smothered' appearance due to the dense accumulation of granules, often distorting the cell's shape.", "Eosinophil cytoplasm demonstrates a 'halo' effect around the nucleus due to the concentration of granules in that region.", 'Lymphocyte cytoplasm is virtually invisible under standard staining, appearing as a thin, delicate rim around the nucleus.', 'Monocyte cytoplasm is irregular and often displays pseudopodia, extending from the cell body.', "Neutrophil cytoplasm is slightly hazy due to the dense packing of granules, creating a 'ground glass' texture."),
("Cytoplasm is densely packed with dark purple granules, often obscuring the nucleus and creating a 'smudge' appearance.", 'Cytoplasm is abundant and filled with large, orange-red granules exhibiting a central crystalline structure.', 'Cytoplasm is scant and clear, with a high nuclear-cytoplasmic ratio and a smooth texture.', 'Cytoplasm is broad, grey-blue, and contains numerous fine azurophilic granules.', 'Cytoplasm is pale pink and densely packed with small, neutral granules.'),
('Granules are irregularly shaped, ranging from 0.5-2 μm in diameter, and intensely stain with basic dyes, exhibiting metachromasia.', 'Granules are uniformly sized (1-3 μm), spherical, and intensely eosinophilic, often appearing as bright orange-red inclusions.', 'Granules are virtually absent, resulting in a clear, homogenous cytoplasm with a smooth texture.', 'Granules are sparse, fine, and azurophilic, appearing as a faint greyish-purple dusting throughout the cytoplasm (approximately 0.2-0.5 μm).', 'Granules are numerous (5-10 per cell), small (0.5-1 μm), and evenly dispersed, giving the cytoplasm a slightly granular texture.'),
("Basophil cytoplasm is filled with irregularly shaped granules that often coalesce, creating a 'mottled' appearance and obscuring the underlying cytoplasmic structure.", 'Eosinophil cytoplasm is densely packed with bright orange-red granules that exhibit a peripheral distribution and a smooth, rounded morphology.', 'Lymphocyte cytoplasm is minimal, appearing as a thin, clear rim around the nucleus; granules are absent, and the cell is typically 7-10 μm in diameter.', "Monocyte cytoplasm is abundant, grey-blue, and contains numerous fine, azurophilic granules that are evenly dispersed throughout, giving a 'dusty' appearance.", "Neutrophil cytoplasm is pale pink and contains numerous, small, neutral granules that create a 'sandblasted' texture."),
('Cytoplasm is filled with irregular, dark purple granules that often coalesce, partially obscuring the nucleus and exhibiting a marked metachromatic shift upon staining.', 'Cytoplasm is densely packed with large, intensely orange-red granules exhibiting a glassy inclusion and a distinct, refractile quality.', 'Cytoplasm is minimal, appearing as a thin, clear halo around a large, centrally located, round nucleus.', 'Cytoplasm is abundant, grey-blue, and contains numerous fine, dust-like azurophilic granules and occasional prominent vacuoles, creating a "ground glass" appearance.', 'Cytoplasm is pale pink and contains numerous, fine lilac granules that are evenly distributed and surround a distinctly segmented nucleus.'),
('Cytoplasm-Color is deep purple-black due to the high concentration of metachromatic granules, often masking the nucleus.', 'Cytoplasm-Color is bright orange-red due to the abundance of eosinophilic granules, creating a vibrant hue.', 'Cytoplasm-Color is pale blue with minimal staining, appearing almost transparent.', 'Cytoplasm-Color is grey-blue with a slight granular appearance and occasional vacuoles.', 'Cytoplasm-Color is pale pink with a granular texture and evenly distributed granules.'),
('Cytoplasm exhibits a homogenous, intensely stained purple-blue appearance due to the dense concentration of granules.', 'Cytoplasm is a vibrant orange-red, reflecting the high concentration of eosinophilic granules, and has a slightly granular texture.', 'Cytoplasm is a pale, almost transparent blue, with minimal staining and a smooth, uniform texture.', 'Cytoplasm is a greyish-blue, with a slightly granular texture and occasional vacuoles, appearing somewhat translucent.', 'Cytoplasm is a pale pink, with a fine granular texture and a slightly translucent appearance.'),
('Granules are irregular in shape and size, often coalescing to form larger, amorphous masses, and exhibit metachromatic staining.', 'Granules are uniformly sized, spherical, and highly refractile, appearing as bright orange-red inclusions within the cytoplasm.', 'Granules are absent, resulting in a clear, sky-blue cytoplasm and a prominent, centrally located nucleus.', 'Granules are small, lilac-colored, and sparsely distributed, appearing as a fine dust-like pattern throughout the cytoplasm.', 'Granules are numerous, pale lilac, and evenly dispersed, with a diameter of approximately 0.5-1 μm, and are non-specific.'),
('Granules are large, dark blue-purple, and often form clusters, obscuring the nucleus.', 'Granules are bright orange-red, with a crystalline core, and are concentrated in the cytoplasm.', 'Cytoplasm is minimal, with a large, densely stained nucleus.', "Cytoplasm is abundant, grey-blue, and displays a 'ground glass' appearance.", 'Granules are small and round, evenly distributed, creating a finely granular texture.'),
('Cytoplasm is intensely basophilic, appearing a deep violet-blue and almost completely obscuring the nucleus with numerous, irregularly shaped metachromatic granules.', 'Cytoplasm exhibits a bright coral-pink hue due to abundant, large eosinophilic granules that often show a central crystalline structure.', 'Cytoplasm is scant and pale blue, presenting a clear, almost empty appearance with a tightly condensed, deeply stained nucleus dominating the cell.', "Cytoplasm is abundant, dull grey-blue, and contains numerous fine, evenly distributed azurophilic granules, giving a 'frosted glass' appearance.", "Cytoplasm is pale peach-colored and densely packed with numerous, fine, lilac-colored granules, creating a 'ground glass' texture; the segmented nucleus is clearly visible."),
('Cell size ranges from 10-12 μm, with granules so numerous they create a solid purple appearance, almost completely masking the nucleus.', 'Cell size is 14-18 μm, with a bilobed nucleus and cytoplasm containing tightly packed, intensely orange-red granules that are highly refractile.', 'Cell size is 6-8 μm, characterized by a high nuclear-cytoplasmic ratio (9:1) and a round, deeply stained nucleus with condensed chromatin and minimal cytoplasm.', 'Cell size is 18-22 μm, exhibiting a kidney-shaped nucleus and abundant, grey-blue cytoplasm with numerous small, clear vacuoles and a fine granular texture.', 'Cell size is 9-11 μm, with a segmented nucleus (3-5 lobes) and pale lilac granules evenly dispersed throughout the cytoplasm, creating a slightly granular appearance.'),
("Basophil granules are irregularly shaped and vary significantly in size, often coalescing to form clumps that distort the cell's outline.", 'Eosinophil granules are uniformly spherical and exhibit a distinct crystalline core, resulting in a bright, refractile appearance under polarized light.', 'Lymphocyte granules are absent, leaving the cytoplasm clear and homogenous, with occasional ribosomal RNA contributing a subtle blue tint.', 'Monocyte cytoplasm contains sparse, small azurophilic granules that appear as faint lilac specks, often accompanied by cytoplasmic pseudopodia.', 'Neutrophil granules are numerous and evenly distributed, displaying a segmented pattern that corresponds to the segmented nucleus.'),
('Cytoplasm exhibits a homogenous, intensely stained purple-blue appearance due to the dense concentration of metachromatic granules.', 'Cytoplasm is a vibrant orange-red, reflecting the high concentration of eosinophilic granules, and has a slightly granular texture.', 'Cytoplasm is a pale, almost transparent blue, with minimal staining and a smooth, uniform texture.', 'Cytoplasm is a greyish-blue, with a slightly granular texture and occasional vacuoles, appearing somewhat translucent.', 'Cytoplasm is a pale pink, with a fine granular texture and a slightly translucent appearance.'),
('Granules are intensely metachromatic, staining dark purple with thionin and blue-black with toluidine blue, often obscuring the nucleus.', 'Granules are uniformly eosinophilic, exhibiting a characteristic crystalline core when viewed under polarized light, and are 2-5 μm in diameter.', 'Granules are absent, resulting in a homogenous, clear cytoplasm with no visible inclusions and a smooth nuclear contour.', 'Granules are sparse, fine, and azurophilic, appearing as a faint greyish-purple dusting, approximately 0.2-0.5 μm in size.', 'Granules are small, neutral, and evenly dispersed, giving the cytoplasm a slightly granular appearance; granules do not obscure the nucleus.'),
('Cytoplasm is densely packed with large, dark purple granules exhibiting metachromasia, often obscuring the nucleus.', 'Cytoplasm contains numerous, uniformly distributed, bright orange-red granules, extending to the cell periphery.', 'Cytoplasm is minimal, with a high nuclear-cytoplasmic ratio and a clear, pale blue appearance, lacking visible granules.', 'Cytoplasm is abundant, grey-blue, and contains numerous fine, azurophilic granules and occasional vacuoles.', 'Cytoplasm is pale pink and filled with numerous, evenly distributed, lilac-colored granules, with a slightly granular texture.'),
('Granules are intensely stained and exhibit a metachromatic effect, shifting color with different stains.', 'Granules are uniform in size and shape, exhibiting a characteristic rod-shaped morphology.', 'Granules are absent, resulting in a homogenous, clear cytoplasm.', 'Granules are fine and dust-like, appearing as a faint haze.', 'Granules are small and round, evenly distributed throughout the cytoplasm.'),
('Cytoplasm is densely packed with large, dark-purple metachromatic granules, often obscuring the nuclear detail.', 'Cytoplasm contains numerous, large, refractile eosinophilic granules with a distinct core, appearing bright orange-red.', 'Cytoplasm is sparse and clear, with a high nuclear-cytoplasmic ratio and a round, densely stained nucleus.', 'Cytoplasm is abundant, greyish-blue, and contains numerous fine, lilac-colored azurophilic granules.', "Cytoplasm is pale pink and filled with numerous, small, neutral granules, creating a 'sand-like' texture."),
('Cytoplasm exhibits a dark purple hue due to the dense concentration of granules, often obscuring underlying structures.', 'Cytoplasm is a vibrant orange-red, reflecting the high concentration of eosinophilic granules and their crystalline inclusions.', 'Cytoplasm is a pale, almost transparent blue, with minimal staining and a smooth texture.', 'Cytoplasm is a greyish-blue, with a slightly granular texture and a subtle dusting of azurophilic granules.', 'Cytoplasm is a pale pink, with a fine granular texture and a slightly hazy appearance.'),
('Granules demonstrate metachromatic staining, shifting from purple to blue depending on the stain used.', 'Granules are consistently round and uniform, approximately 2-3 μm in diameter, and stain intensely with eosin.', 'Cytoplasm is devoid of granules, presenting a clear and homogenous appearance.', 'Granules are fine and dust-like, appearing as a faint haze within the cytoplasm.', 'Granules are small and round, evenly distributed throughout the cytoplasm, creating a finely granular texture.'),
('Cytoplasm color is a deep, rich purple-black due to the high concentration of metachromatic granules.', 'Cytoplasm color is a vibrant orange-red, reflecting the abundance of eosinophilic granules and their staining properties.', 'Cytoplasm color is a pale, almost transparent blue, with minimal staining and a smooth, homogenous texture.', 'Cytoplasm color is a grey-blue with a slightly granular appearance and numerous small, clear vacuoles.', 'Cytoplasm color is a pale pink with a finely granular texture and evenly distributed, neutrally stained granules.'),
('Granules are intensely basophilic, staining a deep violet color, and are often clustered, creating localized areas of high granule density.', 'Granules are consistently spherical, 3-6 μm in diameter, and intensely eosinophilic, appearing as bright orange-red spheres with a distinct refractive index.', 'Granules are absent, resulting in a clear cytoplasm that allows for easy visualization of the condensed nuclear chromatin.', 'Granules are sparse, fine, and azurophilic, appearing as a faint greyish-purple dusting concentrated near the cell periphery, approximately 1 μm in diameter.', 'Granules are numerous, small (0.2-0.5 μm), and neutrally stained, creating a finely granular cytoplasmic background with a subtle pink hue.'),
('Granules are irregular in shape and size, staining intensely with basic dyes and often forming clusters.', 'Granules are uniform in size and shape, appearing as bright orange-red spheres throughout the cytoplasm.', 'Granules are absent, resulting in a clear cytoplasm and a prominent, deeply stained nucleus.', 'Granules are sparse and fine, appearing as a faint greyish-purple dusting concentrated near the cell periphery.', 'Granules are small and evenly distributed, staining a neutral pink and providing a slightly granular cytoplasmic background.'),
('Cytoplasm is filled with numerous, irregularly shaped granules that stain intensely purple.', 'Cytoplasm is filled with large, eosinophilic granules that exhibit a crystalline core and stain bright orange.', 'Cytoplasm is scant and clear, with few visible granules and a high nuclear-cytoplasmic ratio.', 'Cytoplasm is abundant, greyish-blue, and contains fine azurophilic granules that stain a faint lilac color.', "Cytoplasm is pale pink with numerous, fine, neutrally stained granules that create a 'sand-like' appearance."),
('Basophil granules are 1-2 μm, intensely stained with toluidine blue, and exhibit a marked metachromatic shift to reddish-purple, often obscuring the nucleus.', 'Eosinophil granules are 3-6 μm, spherical, and contain major basic protein, resulting in a bright orange-red color and a refractile quality, often arranged in clusters.', 'Lymphocyte cytoplasm is minimal, appearing as a thin, pale blue halo around a large, round nucleus with condensed chromatin and a smooth nuclear membrane.', "Monocyte cytoplasm is abundant, grey-blue, and contains numerous small, lilac-colored granules and occasional clear vacuoles, creating a 'frosted' appearance.", 'Neutrophil cytoplasm is pale pink with evenly distributed, fine lilac granules and occasional rod-shaped azurophilic granules (Auer rods) in cases of severe infection.'),
('Basophil granules are 1-3 μm in diameter, intensely staining with toluidine blue, and exhibit a marked metachromatic shift to reddish-purple, often obscuring the nucleus.', 'Eosinophil granules are 3-6 μm, spherical, and contain major basic protein, resulting in a bright orange-red color and a refractile quality, often arranged in clusters.', 'Lymphocyte cytoplasm is minimal, appearing as a thin, pale blue halo around a large, round nucleus with condensed chromatin and a smooth nuclear membrane.', "Monocyte cytoplasm is abundant, grey-blue, and contains numerous small, lilac-colored granules and occasional clear vacuoles, creating a 'frosted' appearance.", 'Neutrophil cytoplasm is pale pink with evenly distributed, fine lilac granules and occasional rod-shaped azurophilic granules (Auer rods) in cases of severe infection.'),
('Basophil granules are 1-3 μm in diameter, intensely stained with toluidine blue, and exhibit a marked metachromatic shift to reddish-purple, often obscuring the nucleus.', 'Eosinophil granules are 3-6 μm, spherical, and contain major basic protein, resulting in a bright orange-red color and a refractile quality, often arranged in clusters.', 'Lymphocyte cytoplasm is minimal, appearing as a thin, pale blue halo around a large, round nucleus with condensed chromatin and a smooth nuclear membrane.', "Monocyte cytoplasm is abundant, grey-blue, and contains numerous small, lilac-colored granules and occasional clear vacuoles, creating a 'frosted' appearance.", 'Neutrophil cytoplasm is pale pink with evenly distributed, fine lilac granules and occasional rod-shaped azurophilic granules (Auer rods) in cases of severe infection.'),
('Cytoplasm stains intensely purple-blue, almost completely masking any underlying structures.', 'Cytoplasm is a vibrant orange-red, reflecting the high concentration of eosinophilic granules.', 'Cytoplasm is a pale, almost translucent blue, with minimal staining.', 'Cytoplasm is a greyish-blue, with a slightly granular texture.', 'Cytoplasm is a pale pink, with a fine granular texture.'),
('Basophil granules are 1-3 μm in diameter, intensely staining with toluidine blue, and exhibit a marked metachromatic shift to reddish-purple.', 'Eosinophil granules are 3-6 μm, spherical, and contain major basic protein, resulting in a bright orange-red color and a refractile quality.', 'Lymphocyte cytoplasm is minimal, appearing as a thin, pale blue halo around a large, round nucleus with condensed chromatin.', "Monocyte cytoplasm is abundant, grey-blue, and contains numerous small, lilac-colored granules and occasional clear vacuoles, creating a 'frosted' appearance.", 'Neutrophil cytoplasm is pale pink with evenly distributed, fine lilac granules and occasional rod-shaped azurophilic granules (Auer rods) in cases of severe infection.'),
('Granules are irregularly shaped, ranging from 0.5-2 μm in diameter, and intensely stain with basic dyes, exhibiting metachromasia and often obscuring the nucleus.', 'Granules are uniformly sized (1-3 μm), spherical, and intensely eosinophilic, often appearing as bright orange-red inclusions within the cytoplasm.', 'Cytoplasm is minimal, appearing as a thin rim around the large, round, deeply stained nucleus; granules are absent, and the cell diameter is 6-8 μm.', 'Cytoplasm is abundant, grey-blue, and contains a few scattered, fine azurophilic granules, with a cell diameter of 16-20 μm.', "Cytoplasm is pale pink, with numerous, evenly distributed, small granules that create a 'ground glass' appearance, with a cell diameter of 10-14 μm."),
('Cytoplasm is obscured by a dense concentration of large, dark purple granules exhibiting metachromasia and irregular shapes.', 'Cytoplasm is abundant and filled with large, refractile, orange-red granules, often forming clusters and extending to the cell periphery.', 'Cytoplasm is minimal, with a high nuclear-cytoplasmic ratio and a clear, pale blue appearance, lacking visible granules.', "Cytoplasm is broad, grey-blue, and contains numerous fine azurophilic granules and occasional vacuoles, giving a 'ground glass' appearance.", 'Cytoplasm is pale pink and densely packed with small, neutral granules, creating a finely granular texture and a segmented nucleus.'),
('Granules are large and irregular, exhibiting metachromatic staining and often forming clusters, partially obscuring the nucleus.', 'Granules are uniformly sized, spherical, and intensely eosinophilic, appearing as bright orange-red inclusions within the cytoplasm.', 'Cytoplasm is minimal, appearing as a thin rim around the large, densely stained nucleus; granules are absent.', 'Cytoplasm is abundant, grey-blue, and contains a few sparse, fine azurophilic granules.', "Cytoplasm is pale pink, with numerous, evenly distributed, small granules that create a 'ground glass' appearance."),
('Cytoplasm is a dark violet-purple, with granules exhibiting metachromasia and a tendency to obscure the nucleus.', 'Cytoplasm is a vibrant orange-red, with granules displaying a characteristic crystalline core and a smooth surface.', 'Cytoplasm is a pale, almost transparent blue, with a smooth texture and a high nuclear-cytoplasmic ratio.', 'Cytoplasm is a greyish-blue, with a slightly granular texture and occasional vacuoles, appearing somewhat translucent.', 'Cytoplasm is a pale pink, with a fine granular texture and a slightly translucent appearance.'),
('Cytoplasm is intensely stained with basic dyes, appearing a deep blue-purple due to the abundance of metachromatic granules, often obscuring cytoplasmic details.', 'Cytoplasm is a vibrant orange-red, reflecting the high concentration of eosinophilic granules, often with a glassy appearance.', 'Cytoplasm is scant and clear, with a pale blue tint and minimal staining.', 'Cytoplasm is greyish-blue, with a slightly granular texture and occasional vacuoles.', 'Cytoplasm is pale pink, with a finely granular texture and a uniform distribution of neutral granules.'),
('Cytoplasm is abundant and contains numerous, irregularly shaped granules that stain intensely purple.', 'Cytoplasm is filled with large, eosinophilic granules that exhibit a crystalline core and stain bright orange.', 'Cytoplasm is scant and clear, with few visible granules and a high nuclear-cytoplasmic ratio.', 'Cytoplasm is abundant, greyish-blue, and contains fine azurophilic granules that stain a faint lilac color.', "Cytoplasm is pale pink with numerous, fine, neutrally stained granules that create a 'sand-like' appearance."),
('Granules are irregularly shaped, ranging from 0.5-1.5 μm, and exhibit marked metachromasia, shifting from deep blue to reddish-purple with Giemsa stain, often obscuring the cytoplasm.', 'Granules are consistently spherical, 2-5 μm in diameter, intensely eosinophilic, and display a characteristic crystalline core, creating a brilliant orange-red appearance.', 'Cytoplasm is scant, limited to a thin halo around the densely pyknotic nucleus, typically 7-10 μm in diameter, and stains a pale, almost transparent blue.', 'The nucleus is often horseshoe-shaped or indented, with a delicate chromatin pattern and a prominent nucleolus; cytoplasm is broad, grey-blue, and contains occasional vacuoles.', 'The nucleus is segmented into 3-5 lobes, each connected by thin chromatin strands; granules are 0.3-0.8 μm, neutrally stained, and evenly distributed, giving a pale lilac hue.'),
("Basophil cytoplasm is scant and filled with intensely staining, dark purple granules that often distort the cell's shape and measure 0.5-1.5 μm.", 'Eosinophil cytoplasm is abundant and contains numerous, bright orange-red granules with a central crystalline inclusion, ranging from 2-6 μm.', 'Lymphocyte cytoplasm is minimal and devoid of vacuoles, appearing as a thin rim around the nucleus with a high nuclear-cytoplasmic ratio.', 'Monocyte cytoplasm is abundant and frequently contains prominent vacuoles, sometimes with ingested material, and fine azurophilic granules.', 'Neutrophil cytoplasm may exhibit occasional small, clear vacuoles, but these are less frequent than in monocytes and is filled with lilac granules.'),
('Cytoplasm-Color is deep purple-black due to the high concentration of metachromatic granules, often masking the nucleus; granules are irregularly shaped.', 'Cytoplasm-Color is bright orange-red due to the abundance of eosinophilic granules, creating a vibrant hue; granules are spherical and refractile.', 'Cytoplasm-Color is pale blue with minimal staining, appearing almost transparent; nucleus is large and round.', 'Cytoplasm-Color is grey-blue with a slight granular appearance and occasional vacuoles; granules are fine and lilac.', 'Cytoplasm-Color is pale pink with a granular texture and evenly distributed granules; nucleus is segmented.'),
('Cytoplasm is filled with large, dark purple granules exhibiting metachromasia, often obscuring the nucleus and creating an irregular cell shape.', 'Cytoplasm is abundant and filled with bright orange-red granules that are consistently spherical and exhibit a distinct, refractive crystalline core.', 'Cytoplasm is a narrow, clear halo surrounding a large, hyperchromatic nucleus, lacking visible granules and presenting a high N/C ratio.', "Cytoplasm is greyish-blue, with a slightly granular texture and occasional small, clear vacuoles, giving a 'ground glass' appearance.", 'Cytoplasm is pale pink, densely packed with fine, neutral granules, and exhibits a segmented nucleus with thin chromatin filaments.'),
('Cytoplasm is intensely stained with metachromatic dyes, exhibiting a deep purple-blue hue.', 'Cytoplasm is a vibrant orange-red, reflecting the high concentration of eosinophilic granules.', 'Cytoplasm is a pale, almost transparent blue, with minimal staining.', 'Cytoplasm is a greyish-blue, with a slightly granular texture.', 'Cytoplasm is a pale pink, with a fine granular texture.'),
('Basophil cytoplasm is densely packed with large, dark purple granules exhibiting metachromasia, often coalescing into irregular masses and obscuring the nucleus.', 'Eosinophil cytoplasm contains numerous, uniformly distributed, bright orange-red granules, extending to the cell periphery and exhibiting a refractive quality; granules are 3-5 μm in diameter.', 'Lymphocyte cytoplasm is minimal, with a high nuclear-cytoplasmic ratio and a clear, pale blue appearance, lacking visible granules and displaying a smooth nuclear contour.', "Monocyte cytoplasm is abundant, grey-blue, and contains numerous fine, azurophilic granules and occasional vacuoles, creating a 'smudged' appearance and a heterogeneous cytoplasmic texture.", 'Neutrophil cytoplasm is pale pink and filled with numerous, evenly distributed, lilac-colored granules, with a slightly granular texture and a segmented nucleus typically exhibiting 3-5 lobes.'),
('Cytoplasm-Color is a deep purple-blue due to the high concentration of metachromatic granules.', 'Cytoplasm-Color is a vibrant orange-red, reflecting the eosinophilic granules.', 'Cytoplasm-Color is a pale, almost transparent blue.', 'Cytoplasm-Color is a greyish-blue, with a slightly translucent appearance.', 'Cytoplasm-Color is a pale pink, with a slightly translucent appearance.'),
('Cytoplasm is intensely basophilic, appearing a deep violet-blue and almost completely obscuring the nucleus with numerous, irregularly shaped metachromatic granules.', 'Cytoplasm exhibits a bright coral-pink hue due to abundant, large eosinophilic granules that often show a central crystalline structure and refractile inclusions.', 'Cytoplasm is scant and pale blue, presenting a clear, almost empty appearance with a tightly condensed, deeply stained nucleus dominating the cell.', "Cytoplasm is abundant, dull grey-blue, and contains numerous fine, evenly distributed azurophilic granules, giving a 'frosted glass' appearance and a slightly granular texture.", "Cytoplasm is pale peach-colored and densely packed with numerous, fine, lilac-colored granules, creating a 'ground glass' texture; the segmented nucleus is clearly visible."),
('Cytoplasm stains intensely with basic dyes, exhibiting a deep violet-purple hue due to metachromasia.', 'Cytoplasm is characterized by a bright orange-red color, reflecting the high concentration of eosinophilic granules.', 'Cytoplasm is minimal and pale blue, with a high nuclear-cytoplasmic ratio and few visible granules.', 'Cytoplasm is greyish-blue and contains sparsely distributed, fine azurophilic granules.', 'Cytoplasm is pale pink and packed with small, evenly distributed, neutral granules.'),
('Cytoplasm is filled with numerous, irregularly shaped granules that stain intensely purple, often obscuring the nucleus.', 'Cytoplasm is filled with large, eosinophilic granules that exhibit a crystalline core and stain bright orange.', 'Cytoplasm is scant and clear, with few visible granules and a high nuclear-cytoplasmic ratio.', 'Cytoplasm is abundant, greyish-blue, and contains fine azurophilic granules that stain a faint lilac color.', "Cytoplasm is pale pink with numerous, fine, neutrally stained granules that create a 'ground glass' appearance."),
('Granules display a marked metachromatic shift, changing color from blue to purple depending on the staining angle.', 'Granules are consistently round, approximately 2 μm in diameter, and exhibit a bright, refractile orange-red color.', 'Cytoplasm is minimal, appearing as a thin rim around the large, round, deeply stained nucleus; granules are absent.', 'Cytoplasm is abundant, grey-blue, and contains a few scattered, fine azurophilic granules.', "Cytoplasm is pale pink, with numerous, evenly distributed, small granules that create a 'ground glass' appearance."),
('Basophil cytoplasm is densely packed with large, irregular, metachromatic granules that frequently mask the nucleus, creating a diffuse, smudged appearance.', 'Eosinophil granules are consistently spherical, exhibit a bright eosinophilic staining pattern, and often demonstrate a crystalline core visible at high magnification.', 'Lymphocyte cytoplasm is minimal, clear, and devoid of granules, presenting a sharply defined nuclear border and a high nuclear-cytoplasmic ratio.', "Monocyte cytoplasm is abundant, grey-blue, and contains sparse, fine azurophilic granules alongside occasional vacuoles, giving a slightly 'frosted' appearance.", "Neutrophil cytoplasm is characterized by numerous, fine, pale lilac granules evenly distributed throughout, creating a 'ground glass' texture and surrounding a segmented nucleus."),
('Basophil cell shape is generally round, but can be slightly irregular due to the dense packing of granules.', 'Eosinophil cell shape is typically round to oval, with the cytoplasm often extending around the nucleus.', 'Lymphocyte cell shape is consistently round, reflecting its small size and high nuclear-to-cytoplasmic ratio.', 'Monocyte cell shape is often irregular and amoeboid, with projections of cytoplasm extending from the cell body.', 'Neutrophil cell shape is round to slightly oval, with the segmented nucleus often appearing as distinct lobes.'),
('Granules are densely packed and partially obscure the nucleus, appearing as dark violet masses with a tendency to coalesce, measuring 1-3 μm.', 'Granules are uniformly sized (3-6 μm) and intensely eosinophilic, creating a vibrant orange-red cytoplasmic background and exhibiting a refractive quality.', 'Cytoplasm is virtually absent, resulting in a high nuclear-cytoplasmic ratio (8:1 or greater) and a deeply stained, round nucleus.', 'Cytoplasm is abundant and contains numerous small, clear vacuoles and scattered, faint azurophilic granules (0.2-0.5 μm).', 'Granules are small (0.5-1 μm) and numerous, creating a finely granular appearance; occasional Döhle bodies, pale blue inclusions, may be present.'),
('Granules exhibit a marked metachromatic shift, appearing blue-violet in one stain and reddish-purple in another.', 'Granules are consistently rod-shaped and intensely eosinophilic, often appearing as bright orange segments.', 'Granules are absent, resulting in a clear cytoplasm that highlights the deeply stained, round nucleus.', 'Granules are few, small, and azurophilic, appearing as a faint, greyish-purple dusting within the cytoplasm.', 'Granules are numerous, small, and neutrally stained, creating a finely granular texture throughout the cytoplasm.'),
('Granules are large, dark-blue, and irregularly shaped, often coalescing to obscure the nucleus, exhibiting metachromasia and a variable staining intensity.', 'Granules are bright orange-red, uniform in size, and exhibit a characteristic rod-shaped morphology, containing major basic protein and exhibiting a crystalline core.', 'Cytoplasm is absent, resulting in a clear, homogenous cytoplasm and a high nuclear-cytoplasmic ratio, with a condensed nucleus.', 'Granules are sparse, fine, and azurophilic, appearing as a faint dusting within the cytoplasm, staining a pale lilac color and exhibiting a variable distribution.', 'Granules are small, neutral, and evenly dispersed, giving the cytoplasm a slightly granular texture and a pale pink hue, with a diameter of 0.5-1 μm.'),
('Granules are large and irregular, often exhibiting a metachromatic staining pattern, and frequently obscure the nucleus.', 'Granules are uniform in size and shape, exhibiting a characteristic rod-shaped morphology and a bright orange-red color.', 'Granules are absent, resulting in a homogenous, clear cytoplasm and a high nuclear-cytoplasmic ratio.', 'Granules are fine and dust-like, appearing as a faint haze within a grey-blue cytoplasm.', 'Granules are small and round, evenly distributed throughout the cytoplasm, and stain a pale lilac color.'),
('Cytoplasm is densely packed with metachromatic granules, appearing almost black in stained preparations, and obscuring cellular details.', 'Cytoplasm is a brilliant orange-red, with granules exhibiting a glassy, refractive quality and a consistent spherical morphology.', 'Cytoplasm is minimal and clear, with a pale blue tint and a high nuclear-cytoplasmic ratio, devoid of granules.', "Cytoplasm is greyish-blue, with a slightly granular texture and occasional small, clear vacuoles, creating a 'smudged' appearance.", 'Cytoplasm is pale pink, with a finely granular texture and a uniform distribution of neutral granules, creating a hazy appearance.'),
('Granules are large, irregular, and stain intensely purple with Giemsa stain, often coalescing into larger masses.', 'Granules are round, uniform, and stain a bright orange-red with eosin, exhibiting a crystalline core.', 'Granules are absent, resulting in a clear cytoplasm with a smooth, non-granular texture.', 'Granules are small, sparse, and stain a faint lilac color, appearing as subtle inclusions within the cytoplasm.', 'Granules are small, numerous, and stain a pale pink, creating a finely granular texture.'),
('Basophil cell size is typically 8-10 μm, with a high nuclear-cytoplasmic ratio obscured by the dense granule packing.', 'Eosinophil cell size is 12-15 μm, with a bilobed nucleus connected by a thin chromatin bridge and abundant, intensely orange-red granules.', 'Lymphocyte cell size is 7-9 μm, with a high nuclear-cytoplasmic ratio (approximately 9:1) and a round, deeply stained nucleus.', 'Monocyte cell size is 14-18 μm, with a kidney-shaped nucleus and abundant grey-blue cytoplasm containing numerous small vacuoles.', 'Neutrophil cell size is 10-14 μm, with a segmented nucleus (3-5 lobes) and pale lilac granules that are evenly distributed.'),
("Basophil nucleus is often hidden by a dense, dark purple granular mass, exhibiting a 'smudge cell' appearance and lacking distinct lobulation.", 'Eosinophil granules are consistently bright orange-red, displaying a central crystalline core and a tendency to cluster near the cell membrane.', 'Lymphocyte nucleus is hyperchromatic and occupies nearly the entire cell volume, with a thin rim of pale blue cytoplasm and no discernible granules.', 'Monocyte cytoplasm is abundant, grey-blue, and contains numerous fine, dust-like azurophilic granules, along with prominent, clear vacuoles.', 'Neutrophil nucleus exhibits a segmented pattern of 3-5 lobes connected by thin chromatin strands, with pale lilac granules evenly distributed throughout the cytoplasm.'),
('Cell Shape is often irregular due to the clustering of granules.', 'Cell Shape is typically round or oval.', 'Cell Shape is round and compact.', 'Cell Shape is often irregular, resembling a kidney bean.', 'Cell Shape is round or slightly elongated.'),
('Cytoplasm is densely packed with large, dark purple granules that often obscure the nucleus, exhibiting metachromasia with varying stain types.', 'Cytoplasm is filled with large, refractile granules that stain a brilliant orange-red, often displaying a crystalline core.', 'Cytoplasm is scant and pale blue, with a high nuclear-cytoplasmic ratio and a smooth, non-granular texture.', 'Cytoplasm is abundant, grey-blue, and contains numerous fine, azurophilic granules that appear as a subtle dusting, often with a ground-glass appearance.', 'Cytoplasm is pale pink and contains numerous, small lilac granules evenly distributed, creating a finely granular texture with a slightly hazy appearance.'),
('Granule morphology is highly pleomorphic, with variations in size, shape, and staining intensity, ranging from 1-5 μm.', 'Granules are uniformly sized (2-3 μm), spherical, and intensely eosinophilic, often appearing as bright orange-red inclusions within the cytoplasm.', 'Cytoplasm is virtually absent, appearing as a thin rim around the large, densely stained nucleus; granules are absent, and the cell diameter is 7-10 μm.', 'Cytoplasm is abundant, grey-blue, and contains a few sparse, fine azurophilic granules, with a cell diameter of 15-20 μm.', 'Cytoplasm is pale pink and exhibits a finely granular texture due to the presence of small granules, with a diameter of 10-14 μm.'),
('Granule distribution is uneven and clustered, with areas of high density interspersed with clear cytoplasmic regions.', 'Granules are uniformly distributed throughout the cytoplasm, forming a dense, homogenous population of bright orange-red spheres.', 'Granules are completely absent, resulting in a clear, homogenous cytoplasm with no visible texture.', 'Granules are sparsely distributed and appear as a faint, lilac dusting, concentrated near the cell periphery.', 'Granules are evenly dispersed, small, and round, creating a subtle, finely granular texture throughout the cytoplasm.'),
('Cytoplasm is obscured by a dense concentration of large, dark purple granules exhibiting metachromasia and irregular shapes.', 'Cytoplasm exhibits numerous, bright orange-red granules with a crystalline appearance, often displaying Charcot-Leyden crystals.', 'Cytoplasm is minimal, appearing as a narrow rim of pale, slightly bluish hue surrounding a large, round nucleus.', 'Cytoplasm is abundant, grey-blue, and contains numerous fine, lilac-colored dust-like granules and prominent vacuoles.', 'Cytoplasm is pale pink with a uniform distribution of fine, lilac granules; occasional Dohle bodies may be present.'),
('Cytoplasm color is deep blue-purple due to the abundance of metachromatic granules.', 'Cytoplasm color is orange-red due to the presence of eosinophilic granules.', 'Cytoplasm color is pale blue with minimal staining.', 'Cytoplasm color is grey-blue with a slight granular appearance.', 'Cytoplasm color is pale pink with a granular texture.'),
('Cytoplasm color is deep purple-blue due to the abundance of metachromatic granules.', 'Cytoplasm color is orange-red due to the presence of eosinophilic granules.', 'Cytoplasm color is pale blue with minimal staining.', 'Cytoplasm color is grey-blue with a slight granular appearance.', 'Cytoplasm color is pale pink with a granular texture.'),
('Granules are large, irregular, and stain a deep purple-black, often coalescing to obscure the cytoplasm.', 'Granules are numerous, spherical, and exhibit a bright orange-red color with a distinct crystalline core.', 'Granules are absent, resulting in a clear cytoplasm and a high nuclear-cytoplasmic ratio.', 'Granules are sparse, small, and azurophilic, appearing as faint lilac specks within a grey-blue cytoplasm.', 'Granules are numerous, fine, and neutrophilic, imparting a pale lilac hue to the cytoplasm.'),
('Granules are large, dark blue-purple, and often mask the nucleus, exhibiting metachromatic staining.', 'Granules are numerous, bright orange-red, and rod-shaped, filling much of the cytoplasm.', 'Granules are absent, resulting in a clear, sky-blue cytoplasm and a centrally located nucleus.', 'Granules are small, lilac-colored, and sparsely distributed, appearing as a fine dust.', 'Granules are numerous, pale lilac, and evenly dispersed, with a diameter of approximately 0.5-1 μm.'),
("Basophil granules are irregularly shaped and intensely basophilic, often coalescing to form large clumps that distort the cell's morphology.", 'Eosinophil granules are consistently round and exhibit a distinct halo around a central crystalline inclusion, staining a bright orange-red and frequently accumulating near the cell periphery.', 'Lymphocyte cytoplasm is minimal and clear, with a smooth texture and lacking any visible granules, emphasizing the large, condensed nucleus.', "Monocyte cytoplasm is broad and contains a moderate number of fine azurophilic granules, interspersed with prominent, clear vacuoles that give the cytoplasm a 'frosted' appearance.", 'Neutrophil cytoplasm is pale pink and contains numerous, evenly distributed lilac granules, creating a finely granular texture and a segmented nucleus with variable lobe counts.'),
('Granules are large, dark-blue, and irregularly shaped, often coalescing to obscure the nucleus.', 'Granules are bright orange-red, uniform in size, and exhibit a characteristic rod-shaped morphology.', 'Granules are absent, resulting in a clear, homogenous cytoplasm.', 'Granules are sparse, fine, and azurophilic, appearing as a faint dusting.', 'Granules are small, neutral, and evenly dispersed, giving the cytoplasm a slightly granular texture.'),
('Granule size is highly variable, ranging from small to large, with irregular shapes.', 'Granules are consistently round and uniform in size, approximately 1-3 μm in diameter.', 'Granules are absent, resulting in a homogenous cytoplasm.', 'Granules are very fine and dust-like, barely visible without high magnification.', 'Granules are small and spherical, evenly dispersed throughout the cytoplasm.'),
]""")
print(forced_double_str)

[('Granules are large, irregular, and exhibit metachromatic staining, shifting from blue to purple depending on the stain and angle of light.', 'Granules are consistently round and uniform, approximately 2-4 μm in diameter, and stain a bright eosinophilic orange-red, appearing highly refractile and crystalline.', 'Cytoplasm is clear and devoid of granules, presenting a smooth, homogenous appearance and a high nuclear-cytoplasmic ratio.', 'Granules are fine and dust-like, appearing as a faint haze within the cytoplasm, staining a pale lilac color and being sparsely distributed.', 'Granules are small and round, evenly distributed throughout the cytoplasm, creating a finely granular texture and a pale pink hue, giving a "ground glass" appearance.'), ('Granules are large, irregular, and exhibit metachromasia, shifting from blue to purple depending on the stain and angle of light.', 'Granules are consistently round and uniform, approximately 2-4 μm in diameter, and stain a bright eosinophil